## Notebook para rodar testar modelos.

# Teste 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from ortools.sat.python import cp_model

In [2]:
alunos = pd.read_excel('../data/processed/matriz_alunos.xlsx', engine='calamine')
aulas = pd.read_excel('../data/processed/matriz_aulas.xlsx', engine='calamine')

In [15]:
alunos.head()

,Aluno,22840,24351,15549,15550,24379,25583,25586,15556,15571,24272,24275,30910,24276,30911,30913,30914,30919,25580
0,Aluno 1,1,1,1,1,0,1,0,1,0,1,1,1,0,0,1,0,0,1
1,Aluno 2,1,0,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1
2,Aluno 3,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
3,Aluno 4,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,Aluno 5,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [16]:
aulas.head()

,DISCIPLINAS,SEGUNDA,TERCA,QUARTA,QUINTA,SEXTA,SABADO,Minima,Maxima
0,22840,0,1,0,0,0,0,20,30
1,15549,1,0,0,0,0,0,20,30
2,15550,0,0,1,0,0,0,20,30
3,24379,0,0,0,0,1,0,20,30
4,25583,0,1,0,0,0,0,20,30


In [37]:
# ===========================================
# OR-Tools (CP-SAT) — lotar matérias em todos os dias + sábado opcional
# - Mantém soft-constraints (Min/Max) com under/over
# - Ignora elegibilidade no sábado (opcional via flag)
# - Nova meta: minimizar "gap até Maxima" (lotar) em todos os dias
# ===========================================
DF_ALUNOS = alunos.copy()
DF_AULAS  = aulas.copy()

col_student = "Aluno"
col_code    = "DISCIPLINAS"
weekday_cols = ["SEGUNDA","TERCA","QUARTA","QUINTA","SEXTA"]
day_cols_all = weekday_cols + ["SABADO"]
col_min     = "Minima"
col_max     = "Maxima"

# ---------- Parâmetros ----------
SATURDAY_IGNORE_ELIGIBILITY = True  # sábado pode ignorar a matriz 0/1
# Pesos do objetivo (ajuste fino conforme sua preferência)
PENAL_MIN_UNDER     = 1000   # faltar ao mínimo (prioridade mais alta)
PENAL_GAP_FILL      = 500    # NÃO preencher até o Max (dias úteis e sábado)
PENAL_SAT_GAP_EXTRA = 100    # peso adicional para lotar sábado (por cima do GAP_FILL)
PENAL_MAX_OVER      = 120    # exceder o máximo
PENAL_NAODISP_WEEK  = 20     # aluno sem disciplina em dias úteis
# sábado é opcional -> sem penalidade para NAO DISPONIVEL no sábado

# ---------- Normalizações ----------
DF_ALUNOS[col_student] = DF_ALUNOS[col_student].astype(str)
code_cols_alunos = [c for c in DF_ALUNOS.columns if c != col_student]
col_map = {str(c).strip(): c for c in code_cols_alunos}

DF_AULAS[col_code] = DF_AULAS[col_code].map(lambda x: str(x).strip())
DF_AULAS[col_min]  = pd.to_numeric(DF_AULAS[col_min], errors="coerce").fillna(0).astype(int)
DF_AULAS[col_max]  = pd.to_numeric(DF_AULAS[col_max], errors="coerce").fillna(0).astype(int)
for d in day_cols_all:
    if d not in DF_AULAS.columns:
        DF_AULAS[d] = 0
    DF_AULAS[d] = pd.to_numeric(DF_AULAS[d], errors="coerce").fillna(0).astype(int)

codes_in = sorted(set(DF_AULAS[col_code].unique()) & set(col_map.keys()))
if not codes_in:
    raise ValueError("Nenhum código em comum entre 'DISCIPLINAS' (aulas) e colunas de 'alunos'.")

DF_AULAS = DF_AULAS[DF_AULAS[col_code].isin(codes_in)].reset_index(drop=True)
students = list(DF_ALUNOS[col_student].astype(str))

# Garantir 0/1 na matriz de elegibilidade
for _, real_col in col_map.items():
    DF_ALUNOS[real_col] = pd.to_numeric(DF_ALUNOS[real_col], errors="coerce").fillna(0).astype(int)

# Tabelas auxiliares
elig    = {(s,c): int(DF_ALUNOS.loc[DF_ALUNOS[col_student]==s, col_map[c]].iloc[0]) for s in students for c in codes_in}
offered = {(c,d): int(DF_AULAS.loc[DF_AULAS[col_code]==c, d].iloc[0]) for c in codes_in for d in day_cols_all}
cap_min = {(c,d): int(DF_AULAS.loc[DF_AULAS[col_code]==c, col_min].iloc[0]) for c in codes_in for d in day_cols_all}
cap_max = {(c,d): int(DF_AULAS.loc[DF_AULAS[col_code]==c, col_max].iloc[0]) for c in codes_in for d in day_cols_all}

# ---------- Modelo ----------
model = cp_model.CpModel()
N = len(students)

# Decisões
x, y = {}, {}
for s in students:
    for d in day_cols_all:
        y[(s,d)] = model.NewBoolVar(f"y_{hash(s)%10**6}_{d}")  # 1 se aluno tem algo no dia d
        for c in codes_in:
            if offered[(c,d)] == 1:
                allowed = 1 if (d == "SABADO" and SATURDAY_IGNORE_ELIGIBILITY) else elig[(s,c)]
                if allowed == 1:
                    x[(s,c,d)] = model.NewBoolVar(f"x_{hash(s)%10**6}_{c}_{d}")

# (A) Ligação y <-> escolhas por dia (no máx. 1 por dia; dias úteis e sábado)
for s in students:
    for d in day_cols_all:
        vars_day = [x[(s,c,d)] for c in codes_in if (s,c,d) in x]
        if vars_day:
            model.Add(sum(vars_day) == y[(s,d)])
            model.Add(sum(vars_day) <= 1)
        else:
            model.Add(y[(s,d)] == 0)

# (B) Capacidade por (c,d) com folgas (soft)
under, over = {}, {}
for c in codes_in:
    for d in day_cols_all:
        if offered[(c,d)] == 1:
            vars_cd = [x[(s,c,d)] for s in students if (s,c,d) in x]
            under[(c,d)] = model.NewIntVar(0, N, f"under_{c}_{d}")  # falta ao mínimo
            over[(c,d)]  = model.NewIntVar(0, N, f"over_{c}_{d}")   # excede o máximo
            model.Add(sum(vars_cd) + under[(c,d)] >= cap_min[(c,d)])
            model.Add(sum(vars_cd) - over[(c,d)]  <= cap_max[(c,d)])

# (C) Gap de lotação até o Máximo (todos os dias)
gap_fill = {}
for c in codes_in:
    for d in day_cols_all:
        if offered[(c,d)] == 1:
            vars_cd = [x[(s,c,d)] for s in students if (s,c,d) in x]
            # gap_fill >= Max - sum(x); gap >= 0
            # (dias úteis: desigualdade; se ultrapassar Max, gap=0 e "over" cobre o excesso)
            gap_fill[(c,d)] = model.NewIntVar(0, cap_max[(c,d)], f"gapfill_{c}_{d}")
            model.Add(sum(vars_cd) + gap_fill[(c,d)] >= cap_max[(c,d)])

# (C.1) Reforço para sábado: igualdade (tenta lotar exatamente até o Max)
sat_gap = {}
for c in codes_in:
    d = "SABADO"
    if offered.get((c,d), 0) == 1:
        vars_cd = [x[(s,c,d)] for s in students if (s,c,d) in x]
        sat_gap[(c,d)] = model.NewIntVar(0, cap_max[(c,d)], f"sat_gap_{c}")
        model.Add(sum(vars_cd) + sat_gap[(c,d)] == cap_max[(c,d)])  # mais forte que a desigualdade

# ---------- Objetivo ----------
obj_terms = []
# 1) cumprir mínimos
obj_terms.append(PENAL_MIN_UNDER * sum(under.values()))
# 2) lotar em todos os dias (gap até o Max)
obj_terms.append(PENAL_GAP_FILL * sum(gap_fill.values()))
# 2b) "empurrão extra" em sábado
if sat_gap:
    obj_terms.append(PENAL_SAT_GAP_EXTRA * sum(sat_gap.values()))
# 3) evitar exceder o máximo
obj_terms.append(PENAL_MAX_OVER * sum(over.values()))
# 4) reduzir NAO DISPONIVEL em dias úteis
obj_terms.append(PENAL_NAODISP_WEEK * sum(1 - y[(s,d)] for s in students for d in weekday_cols))
model.Minimize(sum(obj_terms))

# ---------- Resolver ----------
solver = cp_model.CpSolver()
# solver.parameters.max_time_in_seconds = 30
res = solver.Solve(model)
status_map = {cp_model.OPTIMAL:"OPTIMAL", cp_model.FEASIBLE:"FEASIBLE",
              cp_model.INFEASIBLE:"INFEASIBLE", cp_model.MODEL_INVALID:"MODEL_INVALID",
              cp_model.UNKNOWN:"UNKNOWN"}
print("Status:", status_map.get(res, str(res)))

# ---------- Saídas ----------
out = pd.DataFrame({"Aluno": students})
for d in day_cols_all:
    out[d] = "NAO DISPONIVEL"

for s in students:
    for d in day_cols_all:
        if solver.Value(y[(s,d)]) == 1:
            for c in codes_in:
                if (s,c,d) in x and solver.Value(x[(s,c,d)]) == 1:
                    out.loc[out["Aluno"]==s, d] = c
                    break

rows = []
for c in codes_in:
    for d in day_cols_all:
        if offered[(c,d)] == 1:
            qtd = sum(solver.Value(x[(s,c,d)]) for s in students if (s,c,d) in x)
            u  = solver.Value(under[(c,d)])
            o  = solver.Value(over[(c,d)])
            gap = solver.Value(gap_fill[(c,d)])
            satg = solver.Value(sat_gap[(c,d)]) if (c,d) in sat_gap else None
            rows.append([c, d, int(qtd), cap_min[(c,d)], cap_max[(c,d)], int(u), int(o), int(gap), (None if satg is None else int(satg))])
resumo = pd.DataFrame(rows, columns=["Codigo","Dia","AlunosAtrib","Minima","Maxima","FaltaMin","ExcedeMax","GapFill","GapLotacaoSabado"])\
         .sort_values(["Dia","Codigo"])

display(resumo.head(10))
display(out.head(10))
out.to_excel("../reports/alocacao_final_soft_sabado.xlsx", index=False)
resumo.to_excel("../reports/resumo_capacidades_soft_sabado.xlsx", index=False)


Status: OPTIMAL


,Codigo,Dia,AlunosAtrib,Minima,Maxima,FaltaMin,ExcedeMax,GapFill,GapLotacaoSabado
1,15550,QUARTA,20,20,30,0,0,10,NaN
2,15556,QUARTA,20,20,30,0,0,10,NaN
13,30911,QUARTA,20,20,30,0,0,10,NaN
9,25580,QUINTA,30,20,30,0,0,0,NaN
11,25586,QUINTA,19,20,30,1,0,11,NaN
14,30913,QUINTA,20,20,30,0,0,10,NaN
3,15571,SABADO,28,7,28,0,0,0,0.0
7,24276,SABADO,28,7,28,0,0,0,0.0
16,30919,SABADO,28,7,28,0,0,0,0.0
0,15549,SEGUNDA,30,20,30,0,0,0,NaN


,Aluno,SEGUNDA,TERCA,QUARTA,QUINTA,SEXTA,SABADO
0,Aluno 1,24275,22840,15550,30913,24272,NAO DISPONIVEL
1,Aluno 2,15549,25583,15556,30913,24272,NAO DISPONIVEL
2,Aluno 3,24275,22840,30911,25586,24379,15571
3,Aluno 4,NAO DISPONIVEL,30910,NAO DISPONIVEL,NAO DISPONIVEL,NAO DISPONIVEL,NAO DISPONIVEL
4,Aluno 5,NAO DISPONIVEL,22840,NAO DISPONIVEL,25580,NAO DISPONIVEL,30919
5,Aluno 6,24275,22840,15556,30913,24379,NAO DISPONIVEL
6,Aluno 7,30914,30910,30911,30913,24379,NAO DISPONIVEL
7,Aluno 8,15549,25583,15550,25580,NAO DISPONIVEL,15571
8,Aluno 9,NAO DISPONIVEL,25583,15556,25580,24379,30919
9,Aluno 10,NAO DISPONIVEL,NAO DISPONIVEL,NAO DISPONIVEL,NAO DISPONIVEL,NAO DISPONIVEL,NAO DISPONIVEL


In [ ]:
alunos.columns = alunos.columns.astype(str)

In [ ]:
def validar_saida(alunos, aulas, out, resumo,
                  col_aluno_out="Aluno",
                  col_code_aulas="DISCIPLINAS",
                  dias_semana=("SEGUNDA","TERCA","QUARTA","QUINTA","SEXTA"),
                  dia_sabado="SABADO",
                  str_ndisp="NAO DISPONIVEL",
                  top_n=15):
    """
    Valida a saída gerada pelo solver:
    - Conteúdo do 'out' (códigos válidos/NAO DISPONIVEL; no máximo 1 por dia).
    - Reconta alocações e compara com 'resumo'.
    - Reporta violações de min/max.
    """

    # --------------------------
    # 0) Preparação
    # --------------------------
    dias_all = list(dias_semana) + [dia_sabado]
    assert col_aluno_out in out.columns, f"Coluna '{col_aluno_out}' não encontrada em out."
    for d in dias_all:
        assert d in out.columns, f"Coluna de dia '{d}' não encontrada em out."

    # Códigos válidos segundo 'aulas'
    cod_validos = set(aulas[col_code_aulas].astype(str).str.strip())
    out_chk = out.copy()
    out_chk[col_aluno_out] = out_chk[col_aluno_out].astype(str)

    # --------------------------
    # 1) Checagens no 'out'
    # --------------------------
    problemas = []

    # 1.1) Valores só podem ser código válido ou NAO DISPONIVEL
    for d in dias_all:
        invalidos = out_chk[~out_chk[d].isin(cod_validos | {str_ndisp})][d].unique()
        if len(invalidos) > 0:
            problemas.append(f"[FORMATO] Valores inválidos na coluna {d}: {invalidos[:top_n]}")

    # 1.2) Máximo 1 por dia -> já é garantido pelo modelo; aqui só conferimos unicidade por célula
    # (Nada a fazer extra; se houver múltiplos, apareceria como string única, então tratamos no parser abaixo)

    # --------------------------
    # 2) Recontar alocações a partir do 'out'
    # --------------------------
    # Tabela de capacidades a partir de aulas
    # (apenas para checar; se seu 'resumo' já vem pronto, comparamos com ele no passo 3)
    cap = []
    col_min = "Minima" if "Minima" in aulas.columns else "min"
    col_max = "Maxima" if "Maxima" in aulas.columns else "max"
    for _, r in aulas.iterrows():
        cod = str(r[col_code_aulas]).strip()
        mn  = int(r[col_min])
        mx  = int(r[col_max])
        for d in dias_all:
            offered = int(pd.to_numeric(r.get(d, 0), errors="coerce"))
            if offered == 1:
                cap.append((cod, d, mn, mx))
    cap_df = pd.DataFrame(cap, columns=["Codigo","Dia","Minima","Maxima"])

    # Contagem real a partir do out
    contagem = []
    for d in dias_all:
        # consolidar quantidade por código (ignorando NAO DISPONIVEL)
        qtd_por_cod = out_chk.loc[out_chk[d] != str_ndisp, d].value_counts()
        for cod, qtd in qtd_por_cod.items():
            contagem.append((cod, d, int(qtd)))
    contagem_df = pd.DataFrame(contagem, columns=["Codigo","Dia","AlunosAtrib"])

    # Juntar com capacidades
    check_df = cap_df.merge(contagem_df, on=["Codigo","Dia"], how="left").fillna({"AlunosAtrib": 0})
    check_df["FaltaMin_calc"] = (check_df["Minima"] - check_df["AlunosAtrib"]).clip(lower=0).astype(int)
    check_df["ExcedeMax_calc"] = (check_df["AlunosAtrib"] - check_df["Maxima"]).clip(lower=0).astype(int)

    # --------------------------
    # 3) Comparar com 'resumo' do solver (se existir colunas)
    # --------------------------
    resumo_cols = set(resumo.columns)
    tem_faltas = {"FaltaMin", "ExcedeMax"}.issubset(resumo_cols)
    tem_atrib  = set(["Codigo","Dia","AlunosAtrib","Minima","Maxima"]).issubset(resumo_cols)

    if tem_atrib:
        # confere igualdade de AlunosAtrib entre recontagem e resumo
        r_key = resumo[["Codigo","Dia","AlunosAtrib"]].copy()
        c_key = check_df[["Codigo","Dia","AlunosAtrib"]].copy()
        cmp = r_key.merge(c_key, on=["Codigo","Dia"], suffixes=("_resumo","_calc"), how="outer").fillna(0)
        diffs = cmp[cmp["AlunosAtrib_resumo"] != cmp["AlunosAtrib_calc"]]
        if not diffs.empty:
            problemas.append(f"[CONSISTENCIA] Diferenças na contagem de 'AlunosAtrib' entre resumo e recontagem: {len(diffs)} linhas (mostrando até {top_n})")
            display(diffs.head(top_n))

    if tem_faltas:
        # confere faltas/excessos
        r_fx = resumo[["Codigo","Dia","FaltaMin","ExcedeMax"]].copy()
        c_fx = check_df[["Codigo","Dia","FaltaMin_calc","ExcedeMax_calc"]].copy()
        cmp_fx = r_fx.merge(c_fx, on=["Codigo","Dia"], how="outer").fillna(0)
        dif_falta = cmp_fx[cmp_fx["FaltaMin"] != cmp_fx["FaltaMin_calc"]]
        dif_exced = cmp_fx[cmp_fx["ExcedeMax"] != cmp_fx["ExcedeMax_calc"]]
        if not dif_falta.empty:
            problemas.append(f"[CONSISTENCIA] Diferenças em 'FaltaMin' entre resumo e cálculo: {len(dif_falta)} (mostrando até {top_n})")
            display(dif_falta.head(top_n))
        if not dif_exced.empty:
            problemas.append(f"[CONSISTENCIA] Diferenças em 'ExcedeMax' entre resumo e cálculo: {len(dif_exced)} (mostrando até {top_n})")
            display(dif_exced.head(top_n))

    # --------------------------
    # 4) Relatórios úteis
    # --------------------------
    print(">> Checagem rápida de violação (recontada a partir de 'out'):")
    viol_min = check_df[check_df["FaltaMin_calc"] > 0].sort_values(["Dia","FaltaMin_calc"], ascending=[True, False])
    viol_max = check_df[check_df["ExcedeMax_calc"] > 0].sort_values(["Dia","ExcedeMax_calc"], ascending=[True, False])

    if not viol_min.empty:
        print(f"- Há {len(viol_min)} casos abaixo do mínimo (mostrando até {top_n}):")
        display(viol_min.head(top_n))
    else:
        print("- Nenhuma falta ao mínimo.")

    if not viol_max.empty:
        print(f"- Há {len(viol_max)} casos acima do máximo (mostrando até {top_n}):")
        display(viol_max.head(top_n))
    else:
        print("- Nenhum excesso ao máximo.")

    # Quantos NAO DISPONIVEL por dia (diagnóstico)
    print("\n>> Quantidade de 'NAO DISPONIVEL' por dia (dias úteis e sábado):")
    for d in dias_all:
        qtd_nd = (out_chk[d] == str_ndisp).sum()
        print(f"  {d}: {qtd_nd}")

    # --------------------------
    # 5) Resultado final dos testes
    # --------------------------
    if problemas:
        print("\n==== RESULTADO: Há pontos a revisar ====")
        for msg in problemas[:top_n]:
            print("-", msg)
    else:
        print("\n==== RESULTADO: OK! 'out' e 'resumo' coerentes com as regras e contagens. ====")

    # Retorna DataFrames de apoio se você quiser usá-los depois
    return {
        "cap_df": cap_df,
        "contagem_df": contagem_df,
        "check_df": check_df
    }

# ---- Executar testes ----
artefatos = validar_saida(alunos, aulas, out, resumo, top_n=20)


# Teste 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from ortools.sat.python import cp_model

In [2]:
alunos = pd.read_excel('../data/processed/matriz_alunos_corrigido.xlsx', engine='calamine')
aulas = pd.read_excel('../data/processed/matriz_aulas_corrigida.xlsx', engine='calamine')

In [3]:
alunos.head()

,Aluno,22722_1,22722_2,22722_3,22705_1,22705_2,22717,22749_1,22749_2,22749_3,...,24275,30910,15556,24272,15571,30914,30911,30913,24276,30919
0,Aluno 1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,1,0,0
1,Aluno 2,0,0,0,1,1,1,1,1,1,...,0,0,1,1,0,0,0,1,0,0
2,Aluno 3,1,1,1,1,1,0,1,1,1,...,1,1,1,1,0,1,1,1,1,1
3,Aluno 4,1,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,0,0,0
4,Aluno 5,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
aulas.head()

,DISCIPLINAS,PERIODO,SEGUNDA,TERCA,QUARTA,QUINTA,SEXTA,SABADO,Turma,Minima,Maxima
0,22705_1,1,0,0,0,0,0,1,1,20,30
1,22717,1,0,0,0,1,0,0,1,10,15
2,22721_1,1,0,0,0,0,0,1,1,20,30
3,22722_1,1,0,1,0,0,0,0,1,10,15
4,22749_1,1,0,0,1,0,0,0,1,10,15


In [ ]:
 # ===========================================
# OR-Tools (CP-SAT) — lotar matérias + sábado opcional
# PRIORIDADE: fechar primeiro as matérias sem underline
# NOVO: custo para "abrir" turmas com "_" desnecessariamente
# ===========================================
import pandas as pd, numpy as np, sys
try:
    from ortools.sat.python import cp_model
except ImportError:
    !{sys.executable} -m pip install ortools -q
    from ortools.sat.python import cp_model

DF_ALUNOS = alunos.copy()
DF_AULAS  = aulas.copy()

col_student = "Aluno"
col_code    = "DISCIPLINAS"
weekday_cols = ["SEGUNDA","TERCA","QUARTA","QUINTA","SEXTA"]
day_cols_all = weekday_cols + ["SABADO"]
col_min     = "Minima"
col_max     = "Maxima"

# ---------- Parâmetros ----------
SATURDAY_IGNORE_ELIGIBILITY = True  # sábado pode ignorar elegibilidade 0/1

# Pesos base do objetivo
PENAL_MIN_UNDER_BASE   = 1000   # faltar ao mínimo (peso padrão)
PENAL_GAP_FILL_BASE    = 500    # não preencher até o Max (peso padrão)
PENAL_SAT_GAP_EXTRA    = 100    # empurrão extra para lotar sábado
PENAL_MAX_OVER         = 120    # exceder o máximo
PENAL_NAODISP_WEEK     = 20     # aluno sem disciplina em dias úteis

# >>> Multiplicadores de prioridade p/ matérias SEM "_"
W_MIN_PLAIN  = 3     # faltar ao mínimo em matéria base custa 3x
W_GAP_PLAIN  = 2     # não lotar matéria base custa 2x
# >>> Custo para "abrir" turmas com "_" (evita seções desnecessárias)
PENAL_OPEN_UNDERSCORE = 200

# ---------- Normalizações ----------
DF_ALUNOS[col_student] = DF_ALUNOS[col_student].astype(str)
code_cols_alunos = [c for c in DF_ALUNOS.columns if c != col_student]
col_map = {str(c).strip(): c for c in code_cols_alunos}

DF_AULAS[col_code] = DF_AULAS[col_code].map(lambda x: str(x).strip())
DF_AULAS[col_min]  = pd.to_numeric(DF_AULAS[col_min], errors="coerce").fillna(0).astype(int)
DF_AULAS[col_max]  = pd.to_numeric(DF_AULAS[col_max], errors="coerce").fillna(0).astype(int)
for d in day_cols_all:
    if d not in DF_AULAS.columns:
        DF_AULAS[d] = 0
    DF_AULAS[d] = pd.to_numeric(DF_AULAS[d], errors="coerce").fillna(0).astype(int)

codes_in = sorted(set(DF_AULAS[col_code].unique()) & set(col_map.keys()))
if not codes_in:
    raise ValueError("Nenhum código em comum entre 'DISCIPLINAS' (aulas) e colunas de 'alunos'.")

DF_AULAS = DF_AULAS[DF_AULAS[col_code].isin(codes_in)].reset_index(drop=True)
students = list(DF_ALUNOS[col_student].astype(str))

# Garantir 0/1 na matriz de elegibilidade
for _, real_col in col_map.items():
    DF_ALUNOS[real_col] = pd.to_numeric(DF_ALUNOS[real_col], errors="coerce").fillna(0).astype(int)

# Tabelas auxiliares
elig    = {(s,c): int(DF_ALUNOS.loc[DF_ALUNOS[col_student]==s, col_map[c]].iloc[0]) for s in students for c in codes_in}
offered = {(c,d): int(DF_AULAS.loc[DF_AULAS[col_code]==c, d].iloc[0]) for c in codes_in for d in day_cols_all}
cap_min = {(c,d): int(DF_AULAS.loc[DF_AULAS[col_code]==c, col_min].iloc[0]) for c in codes_in for d in day_cols_all}
cap_max = {(c,d): int(DF_AULAS.loc[DF_AULAS[col_code]==c, col_max].iloc[0]) for c in codes_in for d in day_cols_all}

# ---------- Matéria base (antes do '_') ----------
def base_subject(code: str) -> str:
    s = str(code)
    return s.split("_")[0] if "_" in s else s

# Mapa: base -> lista de códigos (turmas) daquela matéria
codes_by_base = {}
for c in codes_in:
    b = base_subject(c)
    codes_by_base.setdefault(b, []).append(c)

# >>> Flag: sem underline?
is_plain = {c: ("_" not in c) for c in codes_in}

# ---------- Modelo ----------
model = cp_model.CpModel()
N = len(students)

# Decisões
x, y = {}, {}
for s in students:
    for d in day_cols_all:
        y[(s,d)] = model.NewBoolVar(f"y_{hash(s)%10**6}_{d}")  # 1 se aluno tem algo no dia d
        for c in codes_in:
            if offered[(c,d)] == 1:
                allowed = 1 if (d == "SABADO" and SATURDAY_IGNORE_ELIGIBILITY) else elig[(s,c)]
                if allowed == 1:
                    x[(s,c,d)] = model.NewBoolVar(f"x_{hash(s)%10**6}_{c}_{d}")

# (A) Ligação y <-> escolhas por dia (no máx. 1 por dia)
for s in students:
    for d in day_cols_all:
        vars_day = [x[(s,c,d)] for c in codes_in if (s,c,d) in x]
        if vars_day:
            model.Add(sum(vars_day) == y[(s,d)])
            model.Add(sum(vars_day) <= 1)
        else:
            model.Add(y[(s,d)] == 0)

# (B) Capacidade por (c,d) com folgas (soft)
under, over = {}, {}
for c in codes_in:
    for d in day_cols_all:
        if offered[(c,d)] == 1:
            vars_cd = [x[(s,c,d)] for s in students if (s,c,d) in x]
            under[(c,d)] = model.NewIntVar(0, N, f"under_{c}_{d}")  # falta ao mínimo
            over[(c,d)]  = model.NewIntVar(0, N, f"over_{c}_{d}")   # excede o máximo
            model.Add(sum(vars_cd) + under[(c,d)] >= cap_min[(c,d)])
            model.Add(sum(vars_cd) - over[(c,d)]  <= cap_max[(c,d)])

# (C) Gap de lotação até o Máximo (todos os dias)
gap_fill = {}
for c in codes_in:
    for d in day_cols_all:
        if offered[(c,d)] == 1:
            vars_cd = [x[(s,c,d)] for s in students if (s,c,d) in x]
            gap_fill[(c,d)] = model.NewIntVar(0, cap_max[(c,d)], f"gapfill_{c}_{d}")
            model.Add(sum(vars_cd) + gap_fill[(c,d)] >= cap_max[(c,d)])

# (C.1) Reforço para sábado: igualdade (tenta lotar exatamente até o Max)
sat_gap = {}
for c in codes_in:
    d = "SABADO"
    if offered.get((c,d), 0) == 1:
        vars_cd = [x[(s,c,d)] for s in students if (s,c,d) in x]
        sat_gap[(c,d)] = model.NewIntVar(0, cap_max[(c,d)], f"sat_gap_{c}")
        model.Add(sum(vars_cd) + sat_gap[(c,d)] == cap_max[(c,d)])

# (D) Matéria única por aluno na semana (base única)
for s in students:
    for b, codes_of_b in codes_by_base.items():
        vars_sb = []
        for c in codes_of_b:
            for d in day_cols_all:
                if (s,c,d) in x:
                    vars_sb.append(x[(s,c,d)])
        if vars_sb:
            model.Add(sum(vars_sb) <= 1)

# >>> (E) Variável "turma aberta" e custo para abrir turmas com "_"
open_cd = {}
for c in codes_in:
    for d in day_cols_all:
        if offered[(c,d)] == 1:
            open_cd[(c,d)] = model.NewBoolVar(f"open_{c}_{d}")
            vars_cd = [x[(s,c,d)] for s in students if (s,c,d) in x]
            # se alguém é alocado, a turma está aberta
            for v in vars_cd:
                model.Add(v <= open_cd[(c,d)])
            # se estiver aberta, tem pelo menos 1 aluno
            if vars_cd:
                model.Add(sum(vars_cd) >= open_cd[(c,d)])
            else:
                # não há como abrir sem variáveis; force 0
                model.Add(open_cd[(c,d)] == 0)

# ---------- Objetivo ----------
obj_terms = []

# 1) cumprir mínimos (peso maior para matérias sem "_")
min_plain_terms = []
min_other_terms = []
for (c,d), var in under.items():
    if is_plain[c]:
        min_plain_terms.append(var)
    else:
        min_other_terms.append(var)

obj_terms.append(PENAL_MIN_UNDER_BASE * (W_MIN_PLAIN * sum(min_plain_terms) + 1 * sum(min_other_terms)))

# 2) lotar (gap até o máximo) — peso maior nas matérias sem "_"
gap_plain_terms = []
gap_other_terms = []
for (c,d), var in gap_fill.items():
    if is_plain[c]:
        gap_plain_terms.append(var)
    else:
        gap_other_terms.append(var)

obj_terms.append(PENAL_GAP_FILL_BASE * (W_GAP_PLAIN * sum(gap_plain_terms) + 1 * sum(gap_other_terms)))

# 2b) empurrão extra no sábado (permanece igual)
if sat_gap:
    obj_terms.append(PENAL_SAT_GAP_EXTRA * sum(sat_gap.values()))

# 3) evitar exceder o máximo (igual para todos)
obj_terms.append(PENAL_MAX_OVER * sum(over.values()))

# 4) reduzir NAO DISPONIVEL nos dias úteis (igual para todos)
obj_terms.append(PENAL_NAODISP_WEEK * sum(1 - y[(s,d)] for s in students for d in weekday_cols))

# >>> 5) evitar abrir turmas com "_" sem necessidade
open_underscore = [open_cd[(c,d)] for (c,d) in open_cd if ("_" in c)]
if open_underscore:
    obj_terms.append(PENAL_OPEN_UNDERSCORE * sum(open_underscore))

model.Minimize(sum(obj_terms))

# ---------- Resolver ----------
solver = cp_model.CpSolver()
# Ex.: limite opcional de tempo
# solver.parameters.max_time_in_seconds = 30
res = solver.Solve(model)
status_map = {cp_model.OPTIMAL:"OPTIMAL", cp_model.FEASIBLE:"FEASIBLE",
              cp_model.INFEASIBLE:"INFEASIBLE", cp_model.MODEL_INVALID:"MODEL_INVALID",
              cp_model.UNKNOWN:"UNKNOWN"}
print("Status:", status_map.get(res, str(res)))

# ---------- Saídas ----------
out = pd.DataFrame({"Aluno": students})
for d in day_cols_all:
    out[d] = "NAO DISPONIVEL"

for s in students:
    for d in day_cols_all:
        if solver.Value(y[(s,d)]) == 1:
            for c in codes_in:
                if (s,c,d) in x and solver.Value(x[(s,c,d)]) == 1:
                    out.loc[out["Aluno"]==s, d] = c
                    break

rows = []
for c in codes_in:
    for d in day_cols_all:
        if offered[(c,d)] == 1:
            qtd = sum(solver.Value(x[(s,c,d)]) for s in students if (s,c,d) in x)
            u  = solver.Value(under[(c,d)])
            o  = solver.Value(over[(c,d)])
            gap = solver.Value(gap_fill[(c,d)])
            satg = solver.Value(sat_gap[(c,d)]) if (c,d) in sat_gap else None
            opn  = solver.Value(open_cd[(c,d)]) if (c,d) in open_cd else 0
            rows.append([c, d, int(qtd), cap_min[(c,d)], cap_max[(c,d)],
                         int(u), int(o), int(gap), (None if satg is None else int(satg)), int(opn)])

resumo = pd.DataFrame(rows, columns=[
    "Codigo","Dia","AlunosAtrib","Minima","Maxima",
    "FaltaMin","ExcedeMax","GapFill","GapLotacaoSabado","TurmaAberta"
]).sort_values(["Dia","Codigo"])

display(resumo.head(10))
display(out.head(10))
out.to_excel("../reports/alocacao_final_soft_sabado.xlsx", index=False)
resumo.to_excel("../reports/resumo_capacidades_soft_sabado.xlsx", index=False)


Status: OPTIMAL


,Codigo,Dia,AlunosAtrib,Minima,Maxima,FaltaMin,ExcedeMax,GapFill,GapLotacaoSabado,TurmaAberta
1,15550,QUARTA,20,20,30,0,0,10,NaN,1
2,15556,QUARTA,20,20,30,0,0,10,NaN,1
25,22730_2,QUARTA,0,10,15,10,0,15,NaN,0
27,22730_4,QUARTA,0,10,15,10,0,15,NaN,0
29,22730_6,QUARTA,4,10,15,6,0,11,NaN,1
31,22730_8,QUARTA,10,10,15,0,0,5,NaN,1
42,22747_1,QUARTA,10,10,15,0,0,5,NaN,1
43,22747_2,QUARTA,0,10,15,10,0,15,NaN,0
44,22747_3,QUARTA,10,10,15,0,0,5,NaN,1
45,22749_1,QUARTA,10,10,15,0,0,5,NaN,1


,Aluno,SEGUNDA,TERCA,QUARTA,QUINTA,SEXTA,SABADO
0,Aluno 1,24275,30910,15556,30913,24272,23089_1
1,Aluno 2,22760_2,25583,15550,30913,24272,24276
2,Aluno 3,30914,30910,30911,30913,24379,23089_1
3,Aluno 4,22760_2,30910,22754_2,22750_2,NAO DISPONIVEL,22758_2
4,Aluno 5,22760_2,22840,22754_2,22750_2,22737_1,22716_4
5,Aluno 6,24275,30910,15556,30913,24379,30919
6,Aluno 7,30914,30910,30911,30913,24379,22758_2
7,Aluno 8,22760_1,30910,15550,22750_2,22737_1,23089_1
8,Aluno 9,22760_2,30910,15556,22742_2,24379,24276
9,Aluno 10,NAO DISPONIVEL,22722_3,22749_3,NAO DISPONIVEL,NAO DISPONIVEL,22716_4


In [7]:
def checar_materia_repetida(out, col_aluno="Aluno", dias=("SEGUNDA","TERCA","QUARTA","QUINTA","SEXTA","SABADO"), nd="NAO DISPONIVEL"):
    O = out.copy()
    O[col_aluno] = O[col_aluno].astype(str)
    def base(code): 
        s = str(code)
        return s.split("_")[0] if "_" in s else s
    problemas = []
    for _, row in O.iterrows():
        s = row[col_aluno]
        bases = [base(row[d]) for d in dias if row[d] != nd]
        dup = pd.Series(bases).value_counts()
        viol = dup[dup > 1]
        if not viol.empty:
            problemas.append((s, dict(viol)))
    if problemas:
        print(f"⚠️ Encontradas {len(problemas)} violações de 'mesma matéria > 1x/semana'. Exemplo(s):")
        for s, d in problemas[:10]:
            print(" -", s, "→", d)
    else:
        print("✅ Nenhuma matéria repetida por aluno na semana.")
    return problemas

_ = checar_materia_repetida(out)


✅ Nenhuma matéria repetida por aluno na semana.
